In [1]:
import requests
import time
import json

from datetime import datetime
from IPython.display import JSON

from unity_sds_client.unity import Unity
from unity_sds_client.unity_services import UnityServices
from unity_sds_client.resources.job_status import JobStatus

In [2]:
unity = Unity()
unity.set_venue_id("unity-sips-test")
process_service = unity.client(UnityServices.PROCESS_SERVICE)

Please enter your Unity username:  gangl
Please enter your Unity password:  ········


In [3]:
processes = process_service.get_processes()
for process in processes:
    print("Process ID: {}".format(process.id))
    print("Process Title: {}".format(process.title))
    print("Process Abstract: {}".format(process.abstract))
    print("Process Version: {}".format(process.process_version))
    print("")
    print(process)

Process ID: chirp:develop
Process Title: CHIRP Rebinning
Process Abstract: chirp
Process Version: develop

unity_sds_client.resources.Process(
    id="chirp:develop",
    title="CHIRP Rebinning",
    abstract="chirp",
    execution_unit="1",
    immediate_deployment=true,
    job_control_options=['async-execute'],
    keywords="",
    output_transmission=['reference'],
    ows_context_url="https://raw.githubusercontent.com/unity-sds/unity-sps-workflows/mike-gangl-patch-1/sounder_sips/chirp/chirp-rebinning-e2e-workflow-wrapper.test.cwl",
    process_version="develop"
)
Process ID: sbg-preprocess-workflow:1.0.0
Process Title: SBG Preprocess Workflow
Process Abstract: workflow for executing the pre-process workflow via WPS
Process Version: 1.0.0

unity_sds_client.resources.Process(
    id="sbg-preprocess-workflow:1.0.0",
    title="SBG Preprocess Workflow",
    abstract="workflow for executing the pre-process workflow via WPS",
    execution_unit="1",
    immediate_deployment=true,
    jo

In [5]:
f = open('workflowDescriptor.json')
data = json.load(f)
data


{'processDescription': {'process': {'id': 'sbg-preprocess-workflow',
   'title': 'SBG Preprocess Workflow',
   'owsContext': {'offering': {'code': 'http://www.opengis.net/eoc/applicationContext/cwl',
     'content': {'href': 'https://raw.githubusercontent.com/unity-sds/sbg-workflows/main/preprocess/sbg-preprocess-workflow.cwl'}}},
   'abstract': 'workflow for executing the pre-process workflow via WPS',
   'keywords': [],
   'inputs': [{'id': 'input_cmr_collection_name',
     'title': 'CMR Collection Name',
     'formats': [{'mimeType': 'text', 'default': True}]},
    {'id': 'input_cmr_search_start_time',
     'title': 'CMR Start Search Time',
     'formats': [{'mimeType': 'text', 'default': True}]},
    {'id': 'input_cmr_search_stop_time',
     'title': 'CMR Start Stop Time',
     'formats': [{'mimeType': 'text', 'default': True}]},
    {'id': 'input_unity_dapa_api',
     'title': 'DAPA Endpoint to use',
     'formats': [{'mimeType': 'text', 'default': True}]},
    {'id': 'input_unity

In [6]:
process_service.deploy_process(data)

<Response [201]>

In [4]:
data = {
  "mode": "async",
  "response": "document",
  "inputs": [
    {
      "id": "input_cmr_collection_name",
      "data": "C2408009906-LPCLOUD"
    },
    {
      "id": "input_cmr_search_start_time",
      "data": "2024-01-03T13:19:36.000Z"
    },
    {
      "id": "input_cmr_search_stop_time",
      "data": "2024-01-03T13:19:36.000Z"
    },
    {
      "id": "input_unity_dapa_api",
      "data": "https://dxebrgu0bc9w7.cloudfront.net/"
    },
    {
      "id": "input_unity_dapa_client",
      "data": "71894molftjtie4dvvkbjeard0"
    },
    {
      "id": "input_crid",
      "data": "001"
    },
    {
      "id": "output_collection_id",
      "data": " urn:nasa:unity:ssips:TEST1:CHRP_16_DAY_REBIN___1"
    },
    {
      "id": "output_data_bucket",
      "data": "ssips-test-ds-storage-reproc"
    }
  ],
  "outputs": [
    {
      "id": "output",
      "transmissionMode": "reference"
    }
  ]
}

try:
    # unity = Unity()
    # unity.set_venue_id("unity-sips-test")
    #process_service = unity.client(UnityServices.PROCESS_SERVICE)
    my_process = process_service.get_process('sbg-preprocess-workflow:1.0.0')
    # Store Job ID to use in future steps
    job = my_process.execute(data)
    print(job)

    # If the job submission is successful, print a success message along with the returned JOB-ID
    print("\nJob Submission Successful!\nJOB ID: {}\n".format(job.id))

except requests.exceptions.HTTPError as e:
    # An error has occurred, print the error message that was generated
    print(e.reason)

unity_sds_client.resources.Job(
    id="9eaab492-1c1c-4d34-a5d8-319f67cda64e",
    status="",
    inputs=None
)

Job Submission Successful!
JOB ID: 9eaab492-1c1c-4d34-a5d8-319f67cda64e



In [5]:
try:

    job_status = job.get_status()
    
    while job_status == JobStatus.RUNNING:
        print("Status for job \"{}\" ({}): {}".format(job.id, datetime.now().strftime("%H:%M:%S"), job_status.value))
        time.sleep(5)
        job_status = job.get_status()
    
    # Print the job status
    print("\nStatus for job \"{}\" ({}): {}\n".format(job.id, datetime.now().strftime("%H:%M:%S"),job_status.value))
    
except requests.exceptions.HTTPError as e:
    # An error has occurred, print the error message that was generated
    print(e.reason)


Status for job "9eaab492-1c1c-4d34-a5d8-319f67cda64e" (16:11:42): failed

